**Changed runtime to GPU and some codes need updates**

In [ ]:
import numpy as np
import pandas as pd
import os
import spacy
import string
import re
from spacy.symbols import ORTH
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

Import files

In [ ]:
def unpack_dataset():
  ! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz > .None
  ! tar -zxf aclImdb_v1.tar.gz


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
unpack_dataset()

--2023-05-18 04:12:27--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  36.1MB/s    in 2.2s    

2023-05-18 04:12:29 (36.1 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
# unpack dataset
from pathlib import Path
PATH = Path("./aclImdb/")
list(PATH.iterdir()) #not mkdir() but iterdir()

[PosixPath('aclImdb/imdb.vocab'),
 PosixPath('aclImdb/train'),
 PosixPath('aclImdb/test'),
 PosixPath('aclImdb/README'),
 PosixPath('aclImdb/imdbEr.txt')]

In [ ]:
! ls ./aclImdb/train/

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [ ]:
path = PATH/"train/pos/2449_7.txt"
path.read_text()

'when i saw commercials for this i was thinking "NO WHAT HAS NICK AT NITE DONE!" because it was taking up "fresh prince" slots. well, i still love the fresh prince. but george lopez is a surprisingly good show. i love how not-stereotypical benny is. carmen is a pretty good character, its really funny to see how stupid and overemotional she can be sometimes. i feel bad for the guy who plays max, he looks much younger then he actually is! but max is a fun character, and acted well. and yeah, angie is a little stereotypical, but she has her funny moments. ha ha george does have a big head! nah but he can be really good too. funny show! it definitely should be on more often then home improvement.'

Tokenization

In [ ]:
# must clean up the <br space/> from text file
re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)
my_tok = spacy.load('en_core_web_sm')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [ ]:
# use tokenizer
path = PATH/"train/pos/2449_7.txt"
spacy_tok(path.read_text())[:4]


['when', 'i', 'saw', 'commercials']

Vocab to index

In [ ]:
pos_files = list((PATH/"train"/"pos").iterdir())
neg_files = list((PATH/"train"/"neg").iterdir())
all_files = pos_files + neg_files
all_files[:1]

[PosixPath('aclImdb/train/pos/939_10.txt')]

In [ ]:
'''
this code needs an update (5/11/23)
i am reading the text file in ascii that are actually not in ascii format
'''

import codecs
counts = Counter()

for path in all_files:
    # Read the text file using the appropriate encoding
    with codecs.open(path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Update the counter with the tokenized text
    counts.update(spacy_tok(text))

removed any words that are less than 5 letters to remove the fillers such as 'the', 'it', 'is'....

In [ ]:
len(counts.keys())

103163

In [ ]:
for word in list(counts):
  if counts[word] < 5:
    del counts[word]

In [ ]:
len(counts.keys())

33893

In [ ]:
vocab2index = {"PAD": 0, "UNK": 1}
words = ["PAD", "UNK"]
for word in counts:
  vocab2index[word] = len(words)
  words.append(word)

  def decode_sentence_v2(self, path, vocab2index, N):
    with codecs.open(path, "r", encoding="utf-8") as file:
      decoded_sentence = file.read()
    
    return decoded_sentence

In [ ]:
import random
import codecs
def decode_sentense_v2(path, vocab2index, N= 400):
  with codecs.open(path, "r", encoding = "utf-8") as file:
    text = file.read()
    x = spacy_tok(text)
  if len(x) > N:
    start_idx = random.randint(0, len(x) - N)
#understand the code here is important. random.randint. not ranint
    x = x[start_idx: start_idx + N]
  enc = np.zeros(N, dtype = np.int32)
  encl = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
  l = min(N, len(encl))
  enc[N-l:] = encl[: l]
  return enc

In [ ]:
path = PATH/"train/neg/211_4.txt"
decode_sentense_v2(path, vocab2index, N = 400)

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           1,   123,   169,   494,   169,   320,     6, 10649,     8,
        4320,  1234,   301,    13,   150,   304,  1170,   110,  1609,
        1360,   269,  5170,  9522,   269, 13283,     3,  2666,   918,
           3, 23441,    36,   153,    63,   905,     6,     1,    97,
          47, 25088,

In [ ]:
'''
fixed the code to change string to ascii format
'''
class ImdbDataset(Dataset):
  def __init__(self, PATH, train = "train", N = 400):
    self.N = N
    self.path_to_images = PATH/train
    self.pos_files = list((self.path_to_images/"pos").iterdir())
    self.neg_files = list((self.path_to_images/"neg").iterdir())
    self.files = self.pos_files + self.neg_files
    self.y = np.concatenate((np.ones(len(self.pos_files), dtype = int), np.zeros(len(self.neg_files), dtype = int)), axis = 0)
    
  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    path = self.files[idx]
    return decode_sentense_v2(path, vocab2index, self.N), self.y[idx]



In [ ]:
train_ds = ImdbDataset(PATH)
test_ds = ImdbDataset(PATH,"test")

In [ ]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
valid_dl = DataLoader(test_ds, batch_size = batch_size, shuffle = False)

Understanding LSTMs

```
# Input dim is the dimension of the embedding for each word (2 in the example)
# Output dim is the dimension of the hidden layer (4 in this example)
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature).
```



In [ ]:
lstm = nn.LSTM(2, 4, batch_first = True)

The resulting tensor is then reshaped using the view() function. The view() function creates a new tensor with the same data as the original tensor but with a different shape. In this case, the resulting tensor is reshaped to have dimensions of (1, len(inputs), -1), where 1 is the batch size, len(inputs) is the length of the original list, and -1 is inferred from the size of the input tensors to ensure that the total number of elements in the reshaped tensor is the same as the original tensor.

In [ ]:
inputs = [torch.randn(1, 2) for _ in range(5)]
inputs = torch.cat(inputs).view(1, len(inputs), -1)
inputs

tensor([[[-0.3676,  0.2871],
         [ 0.8069,  0.4595],
         [ 0.4303,  1.5000],
         [-0.0108, -0.1190],
         [ 0.6559,  0.5714]]])

```
# RNNs assume this input shape
# input shape should be  bash_size x seq_len x embedding dimension (if batch_first=True)
```

In [ ]:
inputs.shape

torch.Size([1, 5, 2])

In [ ]:
out, hidden = lstm(inputs)

Model 1

In [ ]:
class LSTMModel(torch.nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim):
    super(LSTMModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.embeddings = nn.Embedding(vocab_size, embed_dim, padding_idx = 0)
    self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first = True)
    self.linearOut = nn.Linear(hidden_dim, 1)

#this is forward here
  def forward(self,inputs):
    x = self.embeddings(inputs)
    lstm_out, lstm_h = self.lstm(x)
    x = lstm_out[:, -1]
    x = self.linearOut(x)
    return x, lstm_h

In [ ]:
def train_epochs(model, train_dl, valid_dl, epochs = 10, lr = 0.001):
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optimizer = torch.optim.Adam(parameters, lr = lr)
  for i in range(epochs):
    model.train()
    sum_loss = 0
    total = 0
    for x, y in train_dl:
      x = x.long().cuda()
      y = y.float().cuda()
      y_pred, _ = model(x)
      optimizer.zero_grad()
      loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
      loss.backward()
      optimizer.step()
      sum_loss += loss.item() * y.shape[0]
      total += y.shape[0]
  print("train loss %.3f" % (sum_loss / total))
  test_metrics(model, test_dl)

In [ ]:
def test_metrics(model, test_dl):
  model.eval()
  correct = 0
  total = 0
  sum_loss = 0.0
  for x, y in test_dl:
    x = x.long().cuda()
    y = y.float().cuda().unsqueeze(1)
    y_hat, _ = model(x)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct += (y_pred.float() == y).float().sum()
    total += y.shape[0]
    sum_loss += loss.item() * y.shape[0]
  print("total loss %.3f and test accuracy %.3f" % (sum_loss / total, correct / total))



In [ ]:
vocab_size = len(words)
print(vocab_size)
model = LSTMModel(vocab_size, 50, 100).cuda()

33895


In [ ]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = batch_size)

In [ ]:
train_epochs(model, train_dl, test_dl, epochs = 20, lr = 0.001)

In [ ]:
def save_model(m, p): torch.save(m.state_dict(), p)
def load_model(m, p): m.load_state_dict(torch.load(p))

In [ ]:
p = PATH/"model-1.pth"
save_model(model, p)